In [12]:
import pandas as pd
import numpy as np
import plotly.express as px

In [13]:
top20 = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (13).csv")
top10 = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (12).csv")
top5 = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (11).csv")
win = pd.read_csv(r"C:\Users\mikej\Downloads\pga_historical_outrights (10).csv")

In [21]:
top20['outcome'] = top20['outcome'].str.strip("T")

outcomes = top20[['player_name','outcome']]
outcomes.columns = ['player','outcome']

In [22]:
outcomes.head(2)

,player,outcome
0,"Scheffler, Scottie",1
1,"Schauffele, Xander",18


In [14]:
odds = pd.concat([win,top5,top10,top20])

odds = odds[['player_name','market','open_odds']].pivot(index='player_name',columns='market',values='open_odds').reset_index().round(2)

odds.columns = ['player','odds_20','odds_10','odds_5','odds_win']

odds = odds[['player','odds_20','odds_10','odds_5','odds_win']]

odds.head(2)

,player,odds_20,odds_10,odds_5,odds_win
0,"Aberg, Ludvig",0.44,0.67,0.27,0.08
1,"An, Byeong Hun",0.17,0.34,0.08,0.02


In [15]:
preds = pd.read_csv(r"C:\Users\mikej\Downloads\rbc-heritage_course-adjustment-model.csv")

preds = preds[['player_name','top_20','top_10','top_5','win']].round(2)

preds.columns = ['player','pred_20','pred_10','pred_5','pred_win']

preds.head(2)

,player,pred_20,pred_10,pred_5,pred_win
0,"Scheffler, Scottie",0.80,0.62,0.45,0.17
1,"Schauffele, Xander",0.62,0.41,0.25,0.07


In [16]:
df = pd.merge(odds,preds)[['player','odds_20','pred_20','odds_10','pred_10','odds_5','pred_5','odds_win','pred_win']]

In [17]:
df.head(2)

,player,odds_20,pred_20,odds_10,pred_10,odds_5,pred_5,odds_win,pred_win
0,"Aberg, Ludvig",0.44,0.53,0.67,0.32,0.27,0.18,0.08,0.04
1,"An, Byeong Hun",0.17,0.29,0.34,0.13,0.08,0.06,0.02,0.01


In [18]:
df['diff_20'] = df['pred_20'] - df['odds_20']
df['diff_10'] = df['pred_10'] - df['odds_10']
df['diff_5'] = df['pred_5'] - df['odds_5']
df['diff_win'] = df['pred_win'] - df['odds_win']

In [23]:
df = df.merge(outcomes,on='player')

In [24]:
def in_top20(a):
    if a > 20:
        return "loss"
    return "win"

def in_top10(a):
    if a > 10:
        return "loss"
    return "win"

def in_top5(a):
    if a > 5:
        return "loss"
    return "win"

def in_win(a):
    if a > 1:
        return "loss"
    return "win"

In [27]:
df['outcome'] = df['outcome'].astype(int)

df['outcome_bool_20'] = df['outcome'].apply(in_top20)
df['outcome_bool_10'] = df['outcome'].apply(in_top10)
df['outcome_bool_5'] = df['outcome'].apply(in_top5)
df['outcome_bool_win'] = df['outcome'].apply(in_win)

In [29]:
df

,player,odds_20,pred_20,odds_10,pred_10,odds_5,pred_5,odds_win,pred_win,diff_20,diff_10,diff_5,diff_win,outcome,outcome_bool_20,outcome_bool_10,outcome_bool_5,outcome_bool_win
0,"Aberg, Ludvig",0.44,0.53,0.67,0.32,0.27,0.18,0.08,0.04,0.09,-0.35,-0.09,-0.04,10,win,win,loss,loss
1,"An, Byeong Hun",0.17,0.29,0.34,0.13,0.08,0.06,0.02,0.01,0.12,-0.21,-0.02,-0.01,67,loss,loss,loss,loss
2,"Barnes, Erik",0.04,0.07,0.11,0.02,0.01,0.01,0.00,0.00,0.03,-0.09,0.00,0.00,58,loss,loss,loss,loss
3,"Bezuidenhout, Christiaan",0.17,0.33,0.36,0.16,0.08,0.08,0.02,0.01,0.16,-0.20,0.00,-0.01,28,loss,loss,loss,loss
4,"Bhatia, Akshay",0.17,0.37,0.36,0.19,0.08,0.09,0.02,0.02,0.20,-0.17,0.01,0.00,18,win,loss,loss,loss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,"Tosti, Alejandro",0.04,0.05,0.12,0.02,0.01,0.00,0.00,0.00,0.01,-0.10,-0.01,0.00,33,loss,loss,loss,loss
65,"Van Rooyen, Erik",0.11,0.21,0.25,0.09,0.04,0.04,0.01,0.01,0.10,-0.16,0.00,0.00,33,loss,loss,loss,loss
66,"Woodland, Gary",0.05,0.07,0.13,0.02,0.02,0.01,0.00,0.00,0.02,-0.11,-0.01,0.00,64,loss,loss,loss,loss
67,"Young, Cameron",0.28,0.41,0.50,0.22,0.14,0.11,0.04,0.02,0.13,-0.28,-0.03,-0.02,62,loss,loss,loss,loss


In [30]:
# top20s = df[['player','odds_20','pred_20','diff_20','outcome']].sort_values('odds_20',ascending=False)
# top10s = df[['player','odds_10','pred_10','diff_10','outcome']].sort_values('odds_10',ascending=False)
# top5s = df[['player','odds_5','pred_5','diff_5','outcome']].sort_values('odds_5',ascending=False)
# wins = df[['player','odds_win','pred_win','diff_win','outcome']].sort_values('odds_win',ascending=False)

In [60]:

px.scatter(df,
           x='diff_20',
           y='pred_20',
           hover_name='player',
           color='outcome_bool_20',
           color_discrete_sequence=px.colors.qualitative.Safe,
           trendline = 'ols',trendline_scope='overall',trendline_color_override = 'darkslategrey',
           width=800,
           height=700,
           size='diff_20')#.add_shape(type="line", x0=0, y0=0, x1=1, y1=1)

In [62]:

px.scatter(df,
           x='diff_10',
           y='pred_10',
           hover_name='player',
           color='outcome_bool_10',
           color_discrete_sequence=px.colors.qualitative.Safe,
           trendline = 'ols',trendline_scope='overall',trendline_color_override = 'darkslategrey',
           width=800,
           height=700,
           size='odds_10')#.add_shape(type="line", x0=0, y0=0, x1=1, y1=1)

In [58]:


px.scatter(df,
           x='diff_5',
           y='pred_5',
           hover_name='player',
           color='outcome_bool_5',
           color_discrete_sequence=px.colors.qualitative.Safe,
           trendline = 'ols',trendline_scope='overall',trendline_color_override = 'darkslategrey',
           width=800,
           height=700,
           size='odds_5')#.add_shape(type="line", x0=0, y0=0, x1=1, y1=1)

In [65]:


# top5s['outcome'] = top5s['outcome'].astype('int')

px.scatter(df,
           x='diff_win',
           y='pred_win',
           hover_name='player',
           color='outcome_bool_win',
           color_discrete_sequence=px.colors.qualitative.Safe,
           trendline = 'ols',trendline_scope='overall',trendline_color_override = 'black',
           width=800,
           height=700,)

In [80]:
temp = df.sort_values('outcome')

In [90]:
px.bar(temp.sort_values('outcome'),#,ascending=False),
    x='player',
    y='outcome',
    color='diff_20').add_vline(x='Kim, Si Woo')